In [1]:
from ipywidgets import HBox, VBox, Output, widgets, Image

from IPython.display import display_svg, display

In [2]:
from pygraphviz import AGraph

class IGraph(AGraph):
    def __init__(self, thing=None, filename=None, data=None,
        string=None, handle=None, name='', strict=True,
        directed=False, styles=None, **attr,):
        self.styles = {} if styles is None else styles
        
        if string:
            for s in self.styles:
                string = string.replace(f'[{s}]', f'[{styles[s].to_str()}]')
        super().__init__(thing=thing, filename=filename, data=data, string=string,
                        handle=handle, name=name, strict=strict, directed=directed, **attr)
        
    def copy(self):
        from copy import deepcopy
        res = super().copy()
        res.styles = deepcopy(self.styles)
        return res
        
    def _repr_svg_(self):
        return str(self.draw(prog='dot', format='svg'))
    
    def select_edges(self, chain: str):
        for p in '-,>':
            chain = chain.replace(p, ' ')
        chain = chain.split()
        return [self.get_edge(*e) for e in zip(chain[:-1], chain[1:])]

    def style_edges(self, edges, attrs):
        if isinstance(edges, str):
            edges = self.select_edges(edges)
        if isinstance(attrs, str):
            attrs = self.styles[attrs]
        for e in edges:
            e.attr.update(attrs)
            
    def delete_node_edges(self, node, in_edges=True, out_edges=True):
        """ Delete the node with its edges."""
        if in_edges: self.delete_edges_from(self.in_edges(node))
        if out_edges: self.delete_edges_from(self.out_edges(node))
        self.delete_node(node)
            
class DotStyle(dict):
    def __init__(self, val=None, **kw):
        val = self.from_str(val) if isinstance(val, str) else {} if val is None else val
        assert isinstance(val, dict)
        val.update(kw)
        super().__init__(val)
        
    @staticmethod
    def from_str(val:str):
        import re

        def parse_val(s, sv):
            if sv:
                for op in (int, float, str):
                    try: s = op(sv)
                    except: continue  # if failed - continue for the next casting attempt
                    break  # otherwise - exit with success
            return s   # last cast option -> str always succeeds 

        found = re.findall(r'(\w+)\s*=\s*(?:"([^"]+)"|([^\s]+))', val)
        return {key: parse_val(s, sv) for key, s, sv in found}
    
    def to_str(self):
        return ' '.join([key + '=' + (f'"{v}"' if isinstance(v, str) and ' ' in v 
                             else f'{v}') for key, v in self.items()])
    
    def __iadd__(self, other):
        self.update(other)
        return self
    
    def __add__(self, other):
        res = self.__class__(self)
        res.update(other)
        return res
    
    def __radd__(self, other):
        return self.__class__(other) + self
    

In [3]:
# Prepare styles
def styles():
    resource = DotStyle(shape='rectangle', width=.9, height=.3, fixedsize='true',
                        fontname="Verdana", fontsize=10, penwidth=.5,
                        color=".14 .2 .4", style='filled', fillcolor=".14 0.2 1")
    produce = DotStyle(shape='oval', penwidth=0.5, fontsize=12)
    
    norm_edge = DotStyle(arrowsize=0.7, penwidth=0.5, color='darkgray')
    bold_edge = DotStyle(arrowsize=0.6, penwidth=1.5, color='black')
    war_edge = bold_edge + {'color': "0 0.5 1"}
    lord_edge = bold_edge + {'color': 'blue'}
    
    return locals()


In [4]:
dotsrc = """
digraph "Feaudal Processes" { 
    rankdir=LR
    ranksep=0.4
    newrank=true
    size = "4.5!"
    
    graph [label="Complete" labelloc=t]
    
    edge[norm_edge]
    node[produce] 

    {node[resource] 
        land peasants goods health protection labor
    } 
    
    subgraph cluster_c1 { label = "" color=white
        Reproduce -> peasants;   
        peasants -> Labor;
        {rank = min peasants Labor}
        {rank=same Reproduce }
    }
    
    goods -> Labor -> {land, labor} -> Work -> goods;
    
    goods -> Lord -> {protection, land};
    
    {goods, labor} -> Health -> health;
    {peasants, goods, labor, health} -> Reproduce;
    
    protection -> War -> {health, goods, land, peasants};  
    
        
    {rank=min   peasants Labor Lord  }
    {rank=same  land protection labor Reproduce}
    {rank=same  War Work Health  }
}

"""

G = IGraph(string=dotsrc, styles=styles())
G

In [5]:
!which dot

/c/Users/ilyap/anaconda3/envs/abc/Scripts/dot


In [6]:
gs = [G]
gs.append(G.copy()); g = gs[-1]

g.graph_attr['label'] = 'Basic Scenario' 
lord_goods_chain = g.select_edges('Lord land Work goods Lord')
g.style_edges(lord_goods_chain, g.styles['bold_edge'] + {'color': 'blue'} )

peasant_goods_chain = g.select_edges('peasants Labor labor Work goods Reproduce peasants Reproduce')
g.style_edges(peasant_goods_chain, g.styles['bold_edge'] + {'color': 'black'})


other_edges = set(g.edges()).difference(peasant_goods_chain).difference(lord_goods_chain)
g.style_edges(other_edges, {'color': 'lightgray'})

g

In [7]:
gs.append(gs[-1].copy()); g = gs[-1]

g.graph_attr['label'] = 'Basic War'

g.style_edges(g.select_edges('Lord protection War'), 'lord_edge')
g.style_edges(g.out_edges('War'), 'war_edge')


In [8]:
outs = [Output() for _ in gs]
for o, g in zip(outs, gs):
    with o:
        display(g)

HBox(outs)

In [9]:
class Lord(abe.Agent):
    def init(land, protection, households, greed=0):
        self.land = self.create('land', land)
        self.protection = self.create('protection', protection)
        self.households = households
        
    

NameError: name 'abe' is not defined